<h3> Import the necessary packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from yahoo_historical import Fetcher

<h3> Read the Nasdaq (QQQ) index

In [2]:
ticker = 'QQQ'
start = [2019,1,1]

df = Fetcher(ticker, start).getHistorical()	
#df = df[['Date','Close']]
df.Date = pd.to_datetime(df.Date)

In [3]:
df.shape

(428, 7)

In [4]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,150.990005,155.750000,150.880005,154.880005,153.021332,58576700
1,2019-01-03,152.600006,153.259995,149.490005,149.820007,148.022064,74820200
2,2019-01-04,152.339996,157.000000,151.740005,156.229996,154.355118,74709300
3,2019-01-07,156.619995,158.860001,156.110001,158.089996,156.192795,52059300
4,2019-01-08,159.539993,160.110001,157.199997,159.520004,157.605637,49388700


<H3> create session gain/loss label for prediction

In [5]:
change = df.Close-df.Close.shift(1)
change.head()
df['gain_loss'] = change>0
df.gain_loss = df.gain_loss.astype(int)
df=df.iloc[1:,]
df.head(7)

,Date,Open,High,Low,Close,Adj Close,Volume,gain_loss
1,2019-01-03,152.600006,153.259995,149.490005,149.820007,148.022064,74820200,0
2,2019-01-04,152.339996,157.000000,151.740005,156.229996,154.355118,74709300,1
3,2019-01-07,156.619995,158.860001,156.110001,158.089996,156.192795,52059300,1
4,2019-01-08,159.539993,160.110001,157.199997,159.520004,157.605637,49388700,1
5,2019-01-09,160.139999,161.520004,159.470001,160.820007,158.890045,46491700,1
6,2019-01-10,159.600006,161.369995,158.699997,161.279999,159.344513,38943400,1
7,2019-01-11,160.330002,160.860001,159.789993,160.690002,158.761581,30176600,0


<h3> fundamentals

In [6]:
import pandas as pd
from searchtweets import load_credentials, gen_rule_payload, collect_results

In [7]:
all_tweets=[]

In [8]:
import os
import re
import pickle
rootdir = os.getcwd()

<h3> Read stock market news dataset

In [11]:
all_tweets=[]
regex = re.compile('stock_market.*\.sav')
#'Run.*\.py$'

for root, dirs, files in os.walk(rootdir):
  for file in files:
    if regex.match(file):
        all_tweets.append(pickle.load(open(file,'rb')))
        print(file)
        

stock_market_2019-03-01-2019-03-30.sav
stock_market_2019-04-01-2019-04-29.sav
stock_market_2019-05-01-2019-05-31.sav
stock_market_2019-06-01-2019-06-28.sav
stock_market_2019-07-01-2019-07-31.sav
stock_market_2019-08-01-2019-08-30.sav
stock_market_2019-09-01-2019-09-30.sav
stock_market_2019-10-01-2019-10-31.sav
stock_market_2019-11-01-2019-11-22.sav
stock_market_2019-11-25-2019-12-30.sav


OSError: [Errno 22] Invalid argument

In [ ]:
len(all_tweets)

In [ ]:
#all_tweets[0]

In [ ]:
text=[]
dates=[]
names=[]
for i in range(len(all_tweets)):
    text.append([tweet.all_text for tweet in all_tweets[i]])
    dates.append([tweet.created_at_datetime for tweet in all_tweets[i]])
    names.append([tweet.name for tweet in all_tweets[i]])

In [ ]:

text1 = [item for sublist in text for item in sublist]
news = pd.DataFrame(text1, columns={'text'})
news['Date'] = [item for sublist in dates for item in sublist]
news['name'] = [item for sublist in names for item in sublist]

In [ ]:
news.shape

In [ ]:
news

In [ ]:
news.to_csv('stock_market_news.cav',index=False)

In [ ]:
from datetime import timedelta

In [ ]:
news.Date = pd.to_datetime(news.Date)
news.Date = [a-timedelta(hours=4) for a in news.Date]
news = news.sort_values('Date').reset_index(drop=True)
news.head()

In [ ]:
newsh=news.Date.astype('str')
#print(d)
newsh = [d[d.find(':')-2:d.find(':')] for d in newsh]
news['hour'] = newsh
news.hour = news.hour.astype(int)
news.head()

In [ ]:
news.shape

<h3> Remove news tweeted after 12 pm so that late news is not included

In [ ]:
print(news[news.hour<=12].shape)
print(news[(news.hour<=12) & (news.hour>=8)].shape)
news1 = news[(news.hour<=12) & (news.hour>=8)]
print(news1.shape)

In [ ]:
news1.dtypes

In [ ]:
news1.shape

In [ ]:
import datetime as dt
news1.Date = pd.to_datetime(news1['Date']).dt.floor('d')
news1.head()

<h3> Merge stock market news dataset with stock price dataset

In [ ]:
df = df.merge(news1, on = 'Date')

In [ ]:
df.head()

In [ ]:
df.shape

<h3> Remove URLs and usernames

In [ ]:
drop_words = [list(filter(lambda word: word[0]!='@', text.split())) for text in df.text]
drop_words = [list(filter(lambda word: word[0:4]!='http', text)) for text in drop_words]
drop_words = [' '.join(a) for a in drop_words]

In [ ]:
len(drop_words)

In [ ]:
drop_words[0]

In [ ]:
df.text[0]

In [ ]:
df.text = drop_words

<h3> remove English stop words

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['this']
stopwords.extend(newStopWords)
example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = set(stopwords) 
  
word_tokens = word_tokenize(example_sent.lower()) 

filtered_sentence = [list(filter(lambda word: word not in stop_words, text.split())) for text in df.text]
filtered_sentence = [' '.join(a) for a in filtered_sentence]
filtered_sentence[0]

In [ ]:
len(filtered_sentence)

In [ ]:
df.text=filtered_sentence

In [ ]:
df.head()

In [ ]:
df.shape

<h3> Create TFID Vector

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.text)

In [ ]:
X_train_counts.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)    
X_train_tfidf.shape

In [ ]:
feature_names = count_vect.get_feature_names()
 
#get tfidf vector for first document
#first_document_vector=X_train_tfidf[0]
first_document_vector=X_train_tfidf[0]

#print the scores
ft = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
ft=ft.sort_values(by=["tfidf"],ascending=False)
ft.head(10)

In [ ]:
type(X_train_tfidf)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2,2))
#X = vectorizer.fit_transform(df.text)
#print(vectorizer.get_feature_names())
processed_features = vectorizer.fit_transform(df.text).toarray()

In [ ]:
print(processed_features)

In [ ]:
from sklearn.model_selection import train_test_split
labels = df.gain_loss
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train.min()

In [ ]:
X_train.max()

<H3> Train classifier to predict stock price based on TFID vector

In [ ]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)
text_classifier.fit(X_train, y_train)

In [ ]:
predictions = text_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

In [ ]:
y_test.value_counts()

In [ ]:
predictions = text_classifier.predict(X_train)

In [ ]:
y_train.value_counts()

In [ ]:
print(confusion_matrix(y_train,predictions))
print(accuracy_score(y_train, predictions))